In [1]:
import numpy as np
from scipy.sparse import linalg as la_sparse
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigs
import time
import primme

## Davidson Algorithm

Consider a (Hermitian) matrix $\textbf{H}$ that we wish to diagonalize to obtain the first $K$ eigenpairs

$$\textbf{H}\textbf{v}_i = \lambda_i\textbf{v}_i\,\,\,\,\,\,\,\,\,\,\, i = 1\ldots K$$

We consider some seaerch subspace $\textbf{B} = [\textbf{b}_1, \textbf{b}_2,\ldots,\textbf{b}_L]$ where $L \geq K$. The search vectors must be orthonormal, meaning that $\textbf{b}_i\cdot\textbf{b}_j = \delta_{ij}$, or in matrix form $\textbf{B}^T\textbf{B} = \textbf{1}$. The ansatz here is that we can form the true eigenvectors as a linear combination of the search vectors, i.e.

$$\textbf{v}_i\approx \textbf{x}_i = \sum_{j=1}^L c_{ji}\textbf{b}_j \Rightarrow \textbf{X} = \textbf{B}\textbf{C}$$

These approximations to the true eigenvectors found within the search space are known as Ritz vectors. Since the Ritz vectors are approximate eigenvectors, we evaluate their quality by forming the Rayleigh quotient for the matrix $\textbf{H}$. The Rayleigh quotient is strictly positive quantity (for a positive-definite matrix) that we can motivate with the following. Consider

$$\textbf{H}\textbf{V} - \mathrm{diag}(\lambda_i)\textbf{V} = 0$$

where $\textbf{V} = [\textbf{v}_1, \textbf{v}_2, \ldots, \textbf{v}_K]$. Then, the quantity

$$\textbf{V}^T\textbf{H}\textbf{V} - \mathrm{diag}(\lambda_i)\textbf{V}^T\textbf{V} = 0$$

If we substitute the any approximation for the eigenvectors $\textbf{V}$, in our case $\textbf{X} = \textbf{B}\textbf{C}$, we should expect to not satisfy equality, but lie slightly above it

$$\textbf{X}^T\textbf{H}\textbf{X} - \mathrm{diag}(\lambda_i)\textbf{X}^T\textbf{X} \geq 0$$


$$R(\textbf{X}) = \dfrac{\textbf{X}^T\textbf{H}\textbf{X}}{\textbf{X}^T\textbf{X}} \geq \mathrm{diag}(\lambda_i)$$

Note that writing the Rayleigh quotient is precisely what we would consider application of the variational theorem in quantum mechanics. Using our expansion in the search space for $\textbf{X}$,

$$R(\textbf{X}) = \dfrac{\textbf{C}^T\textbf{B}^T\textbf{H}\textbf{B}\textbf{C}}{\textbf{C}^T\textbf{B}^T\textbf{B}\textbf{C}} = \dfrac{\textbf{C}^T(\textbf{B}^T\textbf{H}\textbf{B})\textbf{C}}{\textbf{C}^T\textbf{C}} \geq \mathrm{diag}(\lambda_i)$$

The quantity in the parentheses is the projection of the original matrix onto the search subspace. It is called the interaction matrix

$$\textbf{G} = \textbf{B}^T\textbf{H}\textbf{B}$$

The Rayleigh quotient can be written as a strictly positive quantity in the form $R(\textbf{C}) = \textbf{C}^T\textbf{G}\textbf{C} - \mathrm{diag}(\lambda_i)\textbf{C}^T\textbf{C} \geq 0$. If we consider a variational optimization of $R(\textbf{C})$, we find

$$\dfrac{dR}{d\textbf{C}^T} = \textbf{G}\textbf{C} - \mathrm{diag}(\lambda_i) \textbf{C} = 0 \Rightarrow \textbf{G}\textbf{C} = \mathrm{diag}(\lambda_i) \textbf{C}$$

Thus, in order to find the best approximation to the true eigenvectors in the search subspace, we must pick a linear combination of search subspace vectors given by the eigenvectors of the interaction matrix!

Because $\textbf{H}$ is too large to store and access in memory, we formulate the algorithm to only access the matrix-vector product ${\sigma}_i = \textbf{H}\textbf{b}_i \Rightarrow {\Sigma} = [\sigma_1, \sigma_2, \ldots, \sigma_L] = \textbf{H}\textbf{B}$, thus

$$\textbf{G} = \textbf{B}^T\Sigma$$

And diagonalizing it to obtain

$$\mathrm{diag}(\omega_i) = \textbf{C}^{-1}\textbf{G}\textbf{C}$$

where $\mathrm{diag}(\omega_i)$ is our variational approximation (upper bound) to the true eigenvalues $\mathrm{diag}(\lambda_i)$. The error of our eigenpairs within the search subspace is evaluated by considering how well they satisfy the original eigenvalue equation

$$\textbf{H}\textbf{V} - \mathrm{diag}(\lambda_i)\textbf{V} = 0 \Rightarrow \textbf{H}\textbf{X} - \mathrm{diag}(\omega_i)\textbf{X} \geq 0$$

$$\textbf{r} = (\textbf{H}-\mathrm{diag}(\omega_i))\textbf{X} = \textbf{H}\textbf{B}\textbf{C} - \mathrm{diag}(\omega_i)\textbf{B}\textbf{C} = \Sigma\textbf{C} - \mathrm{diag}(\omega_i)\textbf{B}\textbf{C}$$

Most everything shown is not actually unique to the Davidson method; it is primarily based on the variational minimization of the Rayleigh quotient which is a starting point for many Krylov-based numerical eigensolvers. The variation from method to method typically lies in how the search space is expanded. The Davidson method is defined by using a diagonal preconditioning of the residue vector to produce

$$\textbf{q}_i = (\textbf{H}_{ii} - \omega_i)^{-1}\textbf{r}\,\,\,\,\,\,\,\,\,\,\, i = 1\ldots K$$

The preconditioned residual vectors are concatenated into a matrix $\textbf{Q} = [\textbf{q}_1, \textbf{q}_2,\ldots,\textbf{q}_K]$ and the search subspace is expanded by appending $\textbf{Q}$ to $\textbf{B}$ so that $\textbf{B} \leftarrow [\textbf{B},\textbf{Q}]$. 

The newly augmented matrix $\textbf{B}$ is then orthonormalized using the Gram-Schmidt procedure, and importantly, vectors that are linearly dependent (defined by having a post-orthonormalization $L_2$-norm less than a pre-defined threshold, say $10^{-5}$) are discarded from the search vector set. This elimination is best performed as part of the Gram-Schmidt procedure.

An important point to address is the initial selection of search vectors. In the vanilla Davidson algorithm, one simply uses unit vectors in the direction of the first $L$ largest diagonal entries of the matrix $\textbf{H}$. This will work very well if the matrix is strongly diagonally dominant. If not, then a more creative initial guess must be used. In quantum chemistry, one might start with a small diagonalization problem in the subspace of relevant 1-, 2-, or generally, n-tuply excited determinants. 

As a final remark, although the Davidson method was formulated for sparse, diagonally-dominant, Hermitian matrices, typically best suited for large-scale configuration interaction (CI) calculations, the algorithm is quite robust and performs well even in the case of non-Hermitian matrices, such as those present in EOMCC calculations (the imaginary parts of the eigenvalues in such cases are numerically negligible). The most important properties of a matrix that is diagonalizable by Davidson is sparsity and strong diagonal dominance. Matrices that are weakly diagonally dominant can be iteratively diagonalized using the Jacobi-Davidson algorithm which uses a different search vector update. It is significantly more work and literature reports tend to show that Davidson works just fine in the majority of quantum chemistry calculations, provided that care is taken in choosing the initial search vector subspace.  

In [2]:
# la_sparse.gmres(A,np.ones(n))

In [3]:
def davidson(A,d,B,omega,maxit,thresh_vec,tol,Lmax):
    
    flag_conv = False
    
    nroot = len(omega)
    
    L = 2*nroot
    #Lmax = n//2

    it = 1
    while it <= maxit:
        
        omega_old = omega.copy()

        L = B.shape[1]
        
        print('Iter-{}    L = {}'.format(it,L))

        SIGMA = np.dot(A,B)

        G = np.dot(B.T,SIGMA)

        omega,alpha = np.linalg.eig(G)
        idx = omega.argsort()[:nroot]
        omega = omega[idx]
        alpha = alpha[:,idx]

        X = np.dot(B,alpha)

        XI_VEC = []
        resid_norm = 0
        
        for j in range(nroot):

            # Calculate residual
            r = np.dot(SIGMA, alpha[:, j]) - omega[j]*X[:,j]
            
            normr = np.linalg.norm(r)
            resid_norm += normr

            if normr > thresh_vec:
                
                # Apply preconditioner
                r = r/(omega[j] - d[j])
                
                XI_VEC.append(r)

            print("    Root {}: e = {:>10.12f} de = {:>10.12f} "
          "|r| = {:>10.12f}".format(j+1, np.real(omega[j]), abs(omega[j] - omega_old[j]), normr))

        # Calculate convergence parameters
        delta_e = np.linalg.norm(omega[:nroot] - omega_old[:nroot])
        resid_per_root = resid_norm/nroot

        # Check convergence
        if resid_per_root < tol and delta_e < tol:
            flag_conv = True
            break
        else:
            if L >= Lmax:
                print('Restarting and collapsing...')
                B = np.dot(B, alpha)
                omega = omega_old.copy()
            else:
                if not len(XI_VEC) == 0:
                    XI_VEC,_ = np.linalg.qr(np.asarray(XI_VEC).T)
                    B = np.concatenate((B,XI_VEC),axis=1)
        it += 1
        
    if flag_conv:
        print('Davidson Converged...')
        print('Eigenvalues:')
        print(omega)
    else:
        print('Davidson failed to converge in {} iterations'.format(maxit))
        
    return omega, X, flag_conv

def check_diagonal_dom(X):
    D = np.diag(np.abs(X)) # Find diagonal coefficients
    S = np.sum(np.abs(X), axis=1) - D # Find row sum without diagonal
    if np.all(D > S):
        print('Matrix is diagonally dominant')
        return True
    else:
        print('Not diagonally dominant')
        return False

In [7]:
n = 1332
sparsity = 10**-4

diagonal = []

ct = 0
while len(diagonal) < n:
    
    ct+=0.5
    
    diagonal.append(ct)
    diagonal.append(ct)
    diagonal.append(ct)
    
    ct+= 0.5
    
    diagonal.append(ct)
    
diagonal = np.asarray(diagonal,dtype=float)

#diagonal = np.diag(np.arange(1,n+1))

A = sparsity*np.random.rand(n,n) + np.diag(diagonal)

check_diagonal_dom(A)

maxit = 100
thresh_vec = 10**-5
tol = 10**-5
Lmax = n//2
nroot = 6
nvec_per_state = 2

d = np.diagonal(A)
idx = np.argsort(d)[:nvec_per_state*nroot]
B = np.eye((n))[:,idx]
omega = d[idx]
omega = omega[:nroot]

omega, X, flag_conv = davidson(A,d,B,omega,maxit,thresh_vec,tol,Lmax)

Matrix is diagonally dominant
Iter-1    L = 12
    Root 1: e = 0.499954386527 de = 0.000094116470 |r| = 0.001338549180
    Root 2: e = 0.500051578397 de = 0.000006752547 |r| = 0.001211830729
    Root 3: e = 0.500163090315 de = 0.000100810130 |r| = 0.003290049064
    Root 4: e = 1.000092226395 de = 0.000000015340 |r| = 0.002104046682
    Root 5: e = 1.499985985711 de = 0.000019120532 |r| = 0.001069264280
    Root 6: e = 1.499985985711 de = 0.000042303670 |r| = 0.001069264280
Iter-2    L = 18
    Root 1: e = -166.577865358267 de = 167.539594650464 |r| = 346.666266118750
    Root 2: e = 0.499954385603 de = 0.000097192794 |r| = 0.000742855604
    Root 3: e = 0.500051577692 de = 0.000111512623 |r| = 0.000685425273
    Root 4: e = 0.500163032901 de = 0.499929193494 |r| = 0.001860193610
    Root 5: e = 1.000092206328 de = 0.499893779401 |r| = 0.001179210040
    Root 6: e = 1.499985986843 de = 0.000008707521 |r| = 0.000625488886
Iter-3    L = 24
    Root 1: e = -378.740667218343 de = 212.18085

    Root 1: e = -2981.388499380308 de = 181.708197037179 |r| = 10177.738361562037
    Root 2: e = -2822.576297569144 de = 278.025444840392 |r| = 8645.549649895629
    Root 3: e = -2699.586364468319 de = 155.295326954368 |r| = 8755.290613694300
    Root 4: e = -2396.939764080313 de = 257.385029192394 |r| = 7403.690276861562
    Root 5: e = -1872.853761637248 de = 280.807523171320 |r| = 5077.655103152133
    Root 6: e = -602.312117095654 de = 118.586893097728 |r| = 1263.769169559529
Iter-20    L = 126
    Root 1: e = -3141.584710187712 de = 162.238596168443 |r| = 10829.012294337328
    Root 2: e = -3069.595215942566 de = 256.620279977000 |r| = 9612.122878798216
    Root 3: e = -2839.745290205545 de = 171.401477667733 |r| = 9508.880144781520
    Root 4: e = -2571.680935596124 de = 185.039393669595 |r| = 8159.023992251051
    Root 5: e = -2034.677152951866 de = 322.625378016260 |r| = 5565.682101434742
    Root 6: e = -518.790421129582 de = 111.676936594465 |r| = 908.977332638775
Iter-21   

KeyboardInterrupt: 

In [ ]:
diagonal = []

ct = 1
while len(diagonal) < n:
    
    diagonal.append(ct)
    diagonal.append(ct)
    diagonal.append(ct)
    
    ct+= 1
    
    diagonal.append(ct)
    
diagonal = np.asarray(diagonal,dtype=float)



In [ ]:
E,V = np.linalg.eig(A)
idx = E.argsort()[:nroot]
E = E[idx]
V = V[:,idx]
print(E)

In [ ]:
for j in range(nroot):
    nm = np.sum( abs(X[:,j] - V[:,j]) )
    print(nm)

In [ ]:
def Amatvec(x):
    return A@x

def Amatmat(x):
    return A@x

Aop = LinearOperator((n,n),matvec = Amatvec, matmat=Amatmat)

omega, evecs = eigs(Aop, k=10,M=None, tol=1e-6, which='SM')

idx = omega.argsort()
omega = omega[idx]
evecs = evecs[:,idx]

In [ ]:
#         Q = np.zeros((n,nroot))

#         resid_norm = 0
#         for j in range(nroot):

# #             r = AX[:,j] - omega[j]*Rz[:,j]
#             r = np.dot(SIGMA,alpha[:,j]) - omega[j]*Rz[:,j]
#             normr = np.linalg.norm(r)
#             resid_norm += normr
            
#             if normr > thresh_vec:
#                 Q[:,j] = r/(omega[j]-d[j])
#             else:
#                 np.delete(Q,j,axis=1)

#             print("    Root {}: e = {:>10.12f} de = {:>10.12f} "
#               "|r| = {:>10.12f}".format(j+1, np.real(omega[j]), abs(omega[j] - omega_old[j]), normr))

In [ ]:
for j in range(nroot):
    nm = np.sum( abs(evecs[:,j] - V[:,j]) )
    print(nm)

In [108]:
# Begin block Davidson routine

def davidson_v2(A,d,maxit,thresh_vec,tol,L,Lmax):

    t0 = time.time()
    
    idx = np.argsort(d)[:L]
    B = np.eye((n))[:,idx]
    omega = d[idx]
    omega = omega[:nroot]

    #L = nroot*2

    it = 1
    while it < maxit:

        omega_old = omega

        B,_ = np.linalg.qr(B)
        L = B.shape[1]

        # Matrix vector (matrix) product
        SIGMA = np.dot(A,B)

        # Create and diagonalize interaction subspace matrix; sort eigenvector/eigenvalue pairs
        G = np.dot(B.T,SIGMA)
        omega,alpha = np.linalg.eig(G)
        idx = omega.argsort()
        omega = omega[idx[:L]][:nroot]
        alpha = alpha[:,idx[:L]][:nroot]
        
        # Ritz vectors (eigenvector approximation)
        Rz = np.dot(B,alpha)
    
        # Residual vector (A*Rz - omega*Rz)
        r = [np.dot(SIGMA,alpha[:,j])-omega[j]*Rz[:,j] for j in range(nroot)]
        
        # Residual norm
        normr = ([np.linalg.norm(r0) for r0 in r])
        resid_norm = sum(normr)
        
        # Subspace expansion array
        Q = np.asarray([r[j]/(omega[j]-d[j]) for j in range(nroot) if normr[j] > thresh_vec],dtype=float)
        Q = Q.T
        
        
        delta_e = np.linalg.norm(omega - omega_old)
        resid_per_root = resid_norm/nroot
        
        print('Iter = {:>2.0f}     L = {:>10.4f}     |r| = {:>10.12f}     de = {:>10.12f}'\
                                                      .format(it,L,resid_per_root,delta_e))

        # Check convergence
        if resid_per_root < tol and delta_e < tol:
            flag_conv = True
            break
        else:
            if L >= Lmax:
                print('Restarting and collapsing...')
                B = np.dot(B, alpha)
                omega = omega_old.copy()
            else:
                B = np.concatenate((B,Q),axis=1)
        it += 1
        
    if flag_conv:
        print('Davidson Converged in {} seconds...'.format(time.time()-t0))
        print('Eigenvalues:')
        print(np.real(omega))
    else:
        print('Davidson failed to converge in {} iterations'.format(maxit))
        

    return omega, Rz, flag_conv, it



In [111]:

nroot = 6
maxit = 100
thresh_vec = 10**-5
tol = 10**-5
L = 2*nroot
Lmax = 200

d = np.diagonal(A)
idx = np.argsort(d)[:L]
B = np.eye((n))[:,idx]
omega = d[idx]
omega = omega[:nroot]
    
omega, Rz, flag_conv, it = davidson_v2(A,d,maxit,thresh_vec,tol,L,Lmax)

/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Iter =  1     L =    12.0000     |r| = 0.001680500703     de = 0.000145675614
Iter =  2     L =    18.0000     |r| = 0.000953165112     de = 0.000000061320
Iter =  3     L =    24.0000     |r| = 0.000719223397     de = 0.000000028775
Iter =  4     L =    30.0000     |r| = 0.000586099404     de = 0.000000018032
Iter =  5     L =    36.0000     |r| = 0.000494933240     de = 0.000000011858
Iter =  6     L =    42.0000     |r| = 0.000412312794     de = 0.000000008000
Iter =  7     L =    48.0000     |r| = 0.000345386010     de = 0.000000006264
Iter =  8     L =    54.0000     |r| = 0.000293190094     de = 0.000000004616
Iter =  9     L =    60.0000     |r| = 0.000251430970     de = 0.000000003490
Iter = 10     L =    66.0000     |r| = 0.000218770023     de = 0.000000003032
Iter = 11     L =    72.0000     |r| = 0.000187652502     de = 0.000000002300
Iter = 12     L =    78.0000     |r| = 0.000156843677     de = 0.000000001261
Iter = 13     L =    84.0000     |r| = 0.000130491386     de = 0

In [112]:
E,V = np.linalg.eig(A)
idx = E.argsort()[:nroot]
E = E[idx]
V = V[:,idx]
print(np.real(E))

for j in range(nroot):
    nm = np.sum( abs(Rz[:,j] - V[:,j]) )
    print(nm)

[0.49995438 0.50005158 0.50016295 1.00009217 1.49998599 1.49998599]
3.5781448602260597e-06
3.831487864185749e-06
6.129234078401617e-06
2.003441581834609
3.01177409194682
3.01177409194682
